<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=03ac633bd8151827cc593a0dd4db9ac7d75cde0abbc4a80823f1d2761e91695a
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f


Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-04 10:41:42
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.45 C
-------------------
Today PnL: -15.22 K (-0.1%)
Current PnL: -19.38 L (-12.7%)
CY Booked + Current PnL: -6.73 L (-4.41%)
-------------------
Total profit:  2.32 L
Total loss:  -21.70 L
-------------------
Total Booked + Current PnL: 20.25 L (16.12%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.71%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 86.37 L (59.49%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.54%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-7.28,68.0,X-LC,5.54,232066.0,21700.0,7078.0,0.20,10.32,3.05,13.68,37.0,3.07,1.61,28.54,XY25,NTT,REFINERIES
78,TTKPRESTIG,770.00,104.48,55.0,M-SC,1.73,88098.0,-12679.0,12774.0,-0.91,-12.58,14.50,0.09,245.0,-0.99,0.61,15.74,OX40N,NTT,DURABLES
79,UNIONBANK,163.00,-5.43,66.0,M-LC,10.15,171289.0,30449.0,14046.0,-0.15,21.62,8.20,31.59,66.0,2.17,1.19,53.83,XY24,NTT,BANKS
33,ICICIGI,2252.93,-12.22,64.0,X-MC,5.26,150886.0,14753.0,15828.0,1.56,10.84,10.49,22.47,91.0,0.93,1.05,25.22,X40,ATH,INSURANCE
2,ABBOTINDIA,35195.00,-10.25,43.0,X-MC,2.48,88680.0,-1998.0,16902.0,1.81,-2.20,19.06,16.44,101.0,-0.12,0.62,17.53,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,REPCOHOME,880.00,-51.56,77.0,H-SC,6.79,278996.0,-7509.0,264851.0,3.04,-2.62,94.93,89.82,134.0,-0.03,1.94,47.61,XY24,NTT,FINANCE
65,SHALBY,327.00,1166.35,54.0,M-SC,19.22,172280.0,-8863.0,54337.0,1.84,-4.89,31.54,25.10,235.0,-0.16,1.20,38.07,XY24,NTT,HEALTHCARE
2,ABBOTINDIA,35195.00,-10.25,43.0,X-MC,2.48,88680.0,-1998.0,16902.0,1.81,-2.20,19.06,16.44,101.0,-0.12,0.62,17.53,X40,ATH,PHARMA
74,TATAMOTORS,1065.00,-51.85,23.0,X-LC,26.39,151371.0,-125152.0,235231.0,1.71,-45.26,155.40,39.81,54.0,-0.53,1.05,6.69,XY24,NTT,AUTO
33,ICICIGI,2252.93,-12.22,64.0,X-MC,5.26,150886.0,14753.0,15828.0,1.56,10.84,10.49,22.47,91.0,0.93,1.05,25.22,X40,ATH,INSURANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TITAGARH,1548.00,-4.73,50.0,H-SC,3.64,203376.0,-38801.0,149563.0,-1.65,-16.02,73.54,45.74,158.0,-0.26,1.41,32.28,XY24,NTT,ENGINEERING
3,ACC,3906.00,-38.07,46.0,X-MC,2.44,184370.0,-53581.0,206236.0,-1.43,-22.52,111.86,64.15,174.0,-0.26,1.28,3.42,XY24,BTT,CEMENT
45,KANSAINER,340.00,-67.97,44.0,H-SC,2.63,221310.0,-48357.0,84695.0,-1.30,-17.93,38.27,13.47,138.0,-0.57,1.54,12.53,XY24,NTT,PAINTS
49,MASFIN,398.61,-16.44,58.0,H-SC,8.82,94845.0,-3135.0,24736.0,-1.26,-3.20,26.08,22.05,152.0,-0.13,0.66,37.86,XR,ATH,FINANCE
16,CAMS,4762.00,-5.16,48.0,X-SC,0.86,254186.0,2330.0,60115.0,-1.18,0.93,23.65,24.79,122.0,0.04,1.76,23.81,X40N,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,26.6,47.0,M-LC,10.71,206734.0,1658.0,110913.0,-1.12,0.81,53.65,54.89,52.0,0.01,1.43,25.1,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,104.48,55.0,M-SC,1.73,88098.0,-12679.0,12774.0,-0.91,-12.58,14.50,0.09,245.0,-0.99,0.61,15.74,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,44.0,H-SC,2.63,221310.0,-48357.0,84695.0,-1.30,-17.93,38.27,13.47,138.0,-0.57,1.54,12.53,XY24,NTT,PAINTS
17,CERA,9475.0,-21.48,56.0,H-SC,1.94,143267.0,-32636.0,74656.0,-0.86,-18.55,52.11,23.89,149.0,-0.44,0.99,23.73,OX40N,NTT,CERAMICS
73,TATAELXSI,9161.0,-24.05,45.0,H-MC,6.03,102695.0,-29140.0,71363.0,-0.53,-22.10,69.49,32.03,98.0,-0.41,0.71,15.03,OX40N,NTT,IT
67,SIS,528.0,2026.42,46.0,H-SC,2.98,85808.0,-25224.0,48833.0,1.02,-22.72,56.91,21.26,156.0,-0.52,0.60,15.93,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-5.43,66.0,M-LC,10.15,171289.0,30449.0,14046.0,-0.15,21.62,8.2,31.59,66.0,2.17,1.19,53.83,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,52.37,63.0,M-MC,9.82,240391.0,15429.0,152216.0,1.18,6.86,63.32,74.53,192.0,0.10,1.67,41.32,XY24,BTT,STEEL
56,RECLTD,446.00,47.87,54.0,M-LC,4.93,207295.0,4125.0,37997.0,-0.61,2.03,18.33,20.74,55.0,0.11,1.44,9.03,XY25,NTT,FINANCE
31,HINDZINC,730.22,26.60,47.0,M-LC,10.71,206734.0,1658.0,110913.0,-1.12,0.81,53.65,54.89,52.0,0.01,1.43,25.10,X5K,ATH,METALS
46,KPIGREEN,731.05,13.63,74.0,H-SC,6.55,135299.0,10002.0,48924.0,-0.35,7.98,36.16,47.03,141.0,0.20,0.94,69.41,MH,ATH,POWER
79,UNIONBANK,163.00,-5.43,66.0,M-LC,10.15,171289.0,30449.0,14046.0,-0.15,21.62,8.20,31.59,66.0,2.17,1.19,53.83,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,13.63,74.0,H-SC,6.55,135299.0,10002.0,48924.0,-0.35,7.98,36.16,47.03,141.0,0.20,0.94,69.41,MH,ATH,POWER
85,WHIRLPOOL,2270.00,-39.02,58.0,M-SC,4.64,126353.0,11555.0,82483.0,-0.36,10.07,65.28,81.92,223.0,0.14,0.88,49.58,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.15,56.0,H-SC,6.08,130703.0,7367.0,124259.0,-0.03,5.97,95.07,106.72,119.0,0.06,0.91,30.75,AR,ATH,MISC
31,HINDZINC,730.22,26.60,47.0,M-LC,10.71,206734.0,1658.0,110913.0,-1.12,0.81,53.65,54.89,52.0,0.01,1.43,25.10,X5K,ATH,METALS
86,WIPRO,420.00,-15.15,42.0,M-LC,5.37,148701.0,-2244.0,111704.0,-0.27,-1.49,75.12,72.51,53.0,-0.02,1.03,4.45,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-51.85,23.0,X-LC,26.39,151371.0,-125152.0,235231.0,1.71,-45.26,155.40,39.81,54.0,-0.53,1.05,6.69,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-40.58,28.0,X-SC,6.32,88827.0,-39843.0,85141.0,-0.03,-30.97,95.85,35.20,219.0,-0.47,0.62,1.95,X40,NTT,FOOTWEAR
44,JSWINFRA,342.00,-21.71,34.0,X-MC,4.84,196391.0,-3956.0,34467.0,0.34,-1.97,17.55,15.23,178.0,-0.11,1.36,29.64,X40N,NTT,REALTY
30,HINDUNILVR,2922.00,-12.79,34.0,X-LC,7.02,244820.0,-12243.0,47373.0,-0.48,-4.76,19.35,13.67,24.0,-0.26,1.70,14.68,XY25,NTT,FMCG
23,DMART,5391.45,-17.72,38.0,X-LC,8.66,209750.0,-657.0,59821.0,0.94,-0.31,28.52,28.12,38.0,-0.01,1.46,23.24,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,ITC,452.00,-37.50,51.0,X-LC,1.16,201590.0,1452.0,19433.0,-0.41,0.73,9.64,10.44,4.0,0.07,1.40,6.33,X40,NTT,FMCG
66,SIEMENS,4671.50,-3.04,47.0,H-LC,1.24,156000.0,-30095.0,77579.0,-0.24,-16.17,49.73,25.51,15.0,-0.39,1.08,14.96,AR,ATH,ELECTRICAL
52,PGHH,17907.65,-30.55,40.0,X-MC,1.29,201165.0,345.0,67451.0,0.15,0.17,33.53,33.76,80.0,0.01,1.40,5.38,X40,ATH,FMCG
28,HAVELLS,2069.16,-6.32,45.0,X-MC,2.90,312480.0,-15511.0,122055.0,-0.62,-4.73,39.06,32.48,92.0,-0.13,2.17,6.16,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-4.67,75.0,X-LC,4.25,249899.0,15755.0,32862.0,0.37,6.73,13.15,20.77,86.0,0.48,1.73,14.11,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.21,39.0,X-SC,37.71,51270.0,-13736.0,164664.0,-0.56,-21.13,321.17,232.18,198.0,-0.08,0.36,0.00,XY24,NTT,MISC
12,BATAINDIA,2096.00,-40.58,28.0,X-SC,6.32,88827.0,-39843.0,85141.0,-0.03,-30.97,95.85,35.20,219.0,-0.47,0.62,1.95,X40,NTT,FOOTWEAR
51,PAGEIND,51769.93,-28.15,38.0,X-MC,7.04,81110.0,-2400.0,22427.0,-0.50,-2.87,27.65,23.98,82.0,-0.11,0.56,2.78,X40,ATH,APPARELS
20,COLPAL,3726.84,-6.08,43.0,X-MC,7.35,219250.0,-44115.0,153431.0,-0.36,-16.75,69.98,41.51,84.0,-0.29,1.52,2.89,XY25,ATH,FMCG
3,ACC,3906.00,-38.07,46.0,X-MC,2.44,184370.0,-53581.0,206236.0,-1.43,-22.52,111.86,64.15,174.0,-0.26,1.28,3.42,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-48.21,39.0,X-SC,37.71,51270.0,-13736.0,164664.0,-0.56,-21.13,321.17,232.18,198.0,-0.08,0.36,0.00,XY24,NTT,MISC
51,PAGEIND,51769.93,-28.15,38.0,X-MC,7.04,81110.0,-2400.0,22427.0,-0.50,-2.87,27.65,23.98,82.0,-0.11,0.56,2.78,X40,ATH,APPARELS
57,RELAXO,1176.00,-40.17,42.0,X-SC,4.46,81498.0,-63662.0,139590.0,-0.22,-43.86,171.28,52.31,136.0,-0.46,0.57,9.09,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.25,43.0,X-MC,2.48,88680.0,-1998.0,16902.0,1.81,-2.20,19.06,16.44,101.0,-0.12,0.62,17.53,X40,ATH,PHARMA
12,BATAINDIA,2096.00,-40.58,28.0,X-SC,6.32,88827.0,-39843.0,85141.0,-0.03,-30.97,95.85,35.20,219.0,-0.47,0.62,1.95,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-29.33,46.0,X-LC,10.08,281361.0,-64595.0,131283.0,-0.78,-18.67,46.66,19.28,1.0,-0.49,1.95,4.01,X40,ATH,IT
39,INFY,2275.00,-18.88,53.0,X-LC,5.37,316412.0,3524.0,168173.0,0.22,1.13,53.15,54.87,3.0,0.02,2.20,9.50,X40,BTT,IT
41,ITC,452.00,-37.50,51.0,X-LC,1.16,201590.0,1452.0,19433.0,-0.41,0.73,9.64,10.44,4.0,0.07,1.40,6.33,X40,NTT,FMCG
83,VBL,671.64,-16.58,53.0,X-LC,7.20,300243.0,-15599.0,128264.0,-0.87,-4.94,42.72,35.67,5.0,-0.12,2.08,8.23,X40N,ATH,FMCG
1,ABB,7934.00,-39.67,53.0,H-LC,5.22,252192.0,-9427.0,128643.0,-0.04,-3.60,51.01,45.57,7.0,-0.07,1.75,7.31,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,42.0,L-SC,28.08,78336.0,-35213.0,75265.0,-1.11,-31.01,96.08,35.27,268.0,-0.47,0.54,91.73,XR,NTT,HOTELS
7,ASIANTILES,137.00,7422.22,69.0,L-SC,9.79,82564.0,-11246.0,87865.0,0.47,-11.99,106.42,81.67,269.0,-0.13,0.57,59.89,XR,NTT,CERAMICS
49,MASFIN,398.61,-16.44,58.0,H-SC,8.82,94845.0,-3135.0,24736.0,-1.26,-3.20,26.08,22.05,152.0,-0.13,0.66,37.86,XR,ATH,FINANCE
85,WHIRLPOOL,2270.00,-39.02,58.0,M-SC,4.64,126353.0,11555.0,82483.0,-0.36,10.07,65.28,81.92,223.0,0.14,0.88,49.58,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.15,56.0,H-SC,6.08,130703.0,7367.0,124259.0,-0.03,5.97,95.07,106.72,119.0,0.06,0.91,30.75,AR,ATH,MISC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,81.27,74.0,H-SC,1.77,157384.0,-25391.0,136452.0,-0.78,-13.89,86.70,60.77,125.0,-0.19,1.09,40.96,XR,NTT,JEWELLERY
59,REPCOHOME,880.00,-51.56,77.0,H-SC,6.79,278996.0,-7509.0,264851.0,3.04,-2.62,94.93,89.82,134.0,-0.03,1.94,47.61,XY24,NTT,FINANCE
46,KPIGREEN,731.05,13.63,74.0,H-SC,6.55,135299.0,10002.0,48924.0,-0.35,7.98,36.16,47.03,141.0,0.20,0.94,69.41,MH,ATH,POWER
62,SAMMAANCAP,326.00,-198.86,77.0,M-SC,32.35,171171.0,21951.0,122233.0,0.14,14.71,71.41,96.62,208.0,0.18,1.19,85.24,XY25,NTT,FINANCE
68,SONACOMS,806.63,-31.67,68.0,M-SC,7.24,85094.0,-16065.0,56068.0,1.04,-15.88,65.89,39.54,202.0,-0.29,0.59,19.99,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.61
1,25,44.02
2,50,75.49


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.16
LC    32.88
MC    22.94
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.08
X40      16.10
X40N     12.43
XY25     11.90
XR        9.93
AR        7.82
OX40N     6.52
X200      1.76
X5K       1.43
SR        1.07
MH        0.94
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.44
X-LC    22.23
X-MC    18.62
M-SC    12.75
X-SC     5.53
M-LC     5.09
H-LC     4.59
H-MC     2.34
M-MC     1.67
L-SC     1.44
L-LC     0.97
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.12,-3.83,37.40
IT,12.36,-17.85,80.98
FINANCE,12.26,-7.04,55.56
MISC,7.80,-15.60,73.92
ELECTRICAL,5.77,-8.62,47.14
PAINTS,5.45,-17.52,35.04
BANKS,4.05,-15.81,83.18
INSURANCE,3.83,-0.27,37.65
AUTO,2.78,-43.39,103.24


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3241689.0,22
XR,1223386.0,13
AR,1159056.0,9
X40,837480.0,12
X40N,676199.0,9
OX40N,543148.0,9
XY25,458818.0,8
SR,266785.0,2
X5K,110913.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3308770.0,24
M-SC,1207926.0,15
X-MC,1191494.0,14
X-LC,1073333.0,13
X-SC,552588.0,6
H-LC,276384.0,3
M-LC,274660.0,4
L-SC,252449.0,3
H-MC,250021.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1159837.0      6
           AR         830798.0      5
           XR         761209.0      7
M-SC       XY24       692577.0      6
X-MC       XY24       574118.0      4
X-LC       X40        464569.0      5
           XY24       305573.0      2
X-SC       X40N       302783.0      4
X-MC       X40        287770.0      6
H-SC       SR         266785.0      2
           OX40N      241217.0      3
X-LC       X40N       220947.0      3
H-LC       AR         206222.0      2
H-MC       XY24       178658.0      1
X-MC       XY25       177137.0      2
X-SC       XY24       164664.0      1
L-SC       XR         163130.0      2
X-MC       X40N       152469.0      2
M-MC       XY24       152216.0      1
M-SC       OX40N      141249.0      4
           XR         129831.0      2
           XY25       122233.0      1
           AR         122036.0      2
M-LC       XR         111704.0      1
           X5K        110913.0      1
L-SC       OX40N       89319.0      1
X-SC       X40         85141.0      1
X-LC       XY25        82244.0      3
H-MC       OX40N       71363.0      1
H-LC       X200        70162.0      1
L-MC       XR          57512.0      1
H-SC       MH          48924.0      1
L-LC       XY25        39207.0      1
M-LC       XY25        37997.0      1
           XY24        14046.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
